In [ ]:
from __future__ import print_function # Python 2/3 compatibility
import os
import glob
import numpy as np
from PIL import Image
import tifffile
from tifffile import imwrite
import numpy
from skimage import io
import matplotlib.pyplot as plt
import pywt

from skimage import (
    data, restoration, util
)

In [ ]:
def get_scaled_prop(max_values):
    """
    Calculates the proportion of each channel based on the provided max_values and rounds it to 3 decimal places.

    Parameters:
        max_values (list or tuple): A list or tuple containing the maximum values for each channel.

    Returns:
        list: A list containing the proportion of each channel, rounded to 3 decimal places.
    """
    # Ensure max_values is a list or tuple and has exactly 3 elements for the RGB channels.
    if not (isinstance(max_values, (list, tuple)) and len(max_values) == 3):
        raise ValueError("max_values must be a list or tuple with exactly 3 elements.")

    # Calculate the sum of max_values to get the normalization factor.
    total = sum(max_values)

    # Avoid division by zero by returning equal proportions if total is 0.
    if total == 0:
        return [1/3, 1/3, 1/3]

    # Calculate the proportion of each channel, round to 3 decimal places and return the results.
    proportions = [round(val / total, 3) for val in max_values]
    return proportions

#takes input a 3D image with RGB channels
#return a list of length 3 with the maximum value for each input channel
#it can be easily changed to mean if required
def getmaxes(marker):
    maxes = []
    for i in range(3):
        cr_channel = marker[:, :, i]
        maxes.append(np.max(cr_channel))

    return maxes

def convert_to_greyscale(image, weights):
    # Define the weights
    weights = np.array(weights)

    # Calculate the weighted sum along the last axis (color channels)
    greyscale_image = np.dot(image, weights)

    # Optional: Convert to desired dtype (if not 'float')
    greyscale_image = greyscale_image.astype(np.uint8)

    return greyscale_image

In [ ]:
#converting a whole folder D2
current_directory = ""
name = 0
# Iterate through all files in the directory
for filename in os.listdir(current_directory):
    # Create a full file path
    cr_img = io.imread(current_directory+'/'+filename)
    gray_markers = []
    for marker in range(6):
        cr_marker = cr_img[marker]
        maxes = getmaxes(cr_marker)
        prop = get_scaled_prop(maxes)
        gray_marker = convert_to_greyscale(cr_marker, prop)
        roll_ball = gray_marker - restoration.rolling_ball(gray_marker)

        # Append filtered and converted image to our list
        gray_markers.append(gray_marker)

    # Stack all grayscale markers into a single array along a new first axis
    final_image = np.stack(gray_markers, axis=0)

    str_name = str(name)+'.tif'
    imwrite(str_name, final_image)
    print("Generated "+str_name+" for "+filename)

    name = name+1